# Türkiye İş Bankası Machine Learning Challenge #4

In [ ]:
import pandas as pd # data processing
import numpy as np # working with arrays
import matplotlib.pyplot as plt # visualization
from termcolor import colored as cl # text customization
import itertools # advanced tools
from sklearn.preprocessing import StandardScaler # data normalization
from sklearn.model_selection import train_test_split # data split
from sklearn.tree import DecisionTreeClassifier # Decision tree algorithm
from sklearn.neighbors import KNeighborsClassifier # KNN algorithm
from sklearn.linear_model import LogisticRegression # Logistic regression algorithm
from sklearn.svm import SVC # SVM algorithm
from sklearn.ensemble import RandomForestClassifier # Random forest tree algorithm
from xgboost import XGBClassifier # XGBoost algorithm
from sklearn.metrics import confusion_matrix # evaluation metric
from sklearn.metrics import accuracy_score # evaluation metric
from sklearn.metrics import f1_score # evaluation metric
import sklearn.metrics as metrics
import math
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import MeanSquaredLogarithmicError
import keras_tuner as kt
import seaborn as sns
from sklearn import metrics
from scipy.special import expit
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import learning_curve
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from numpy import array
import xgboost as xgb
import optuna
import prince
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold, cross_val_score
pd.set_option("display.float_format", lambda x: "%.3f" % x)
train = pd.read_csv("train.csv")
real_test = pd.read_csv("test.csv")


# Data PreProcessing

In [ ]:
train.TXN_TIME=pd.cut(train.TXN_TIME,bins=[-1,600,1200,1800,2400],labels=['1','2','3','4'])  
train["TXN_TIME"] = train["TXN_TIME"].astype('int')
train.DAY_OF_MONTH=pd.cut(train.DAY_OF_MONTH,bins=[0,7,14,21],labels=['1','2','3'])
train["DAY_OF_MONTH"] = train["DAY_OF_MONTH"].astype('int')
train["DAY_OF_WEEK"] = train["DAY_OF_WEEK"].astype('int')
train["TARGET"] = train["TARGET"].astype('int')
train["TXN_AMNT"] = train["TXN_AMNT"].astype('int')
del train['TXN_TRM']
train.drop_duplicates(inplace=True, ignore_index=True)
train.info()
train.isna().sum() 
train.describe(include='all')

In [ ]:
# Table of TARGET vs. DAY_OF_WEEK
cross_tab = pd.crosstab(index=train["TARGET"], 
                           columns=train["DAY_OF_WEEK"])

cross_tab.index= ["0","1"]
cross_tab

In [ ]:
#categorical data
categorical = ['CST_NR', 'CC_NR', 'TXN_SOURCE', 'TXN_ENTRY','CITY','COUNTRY','MC_NAME','MC_ID','MCC_CODE'] 
for feature in categorical:
  train[feature] = train[feature].mask(train[feature].map(train[feature].value_counts(normalize=True)) < 0.001, 'Other')

#import pandas as pd
train = pd.get_dummies(train, columns = categorical)
train.head()

In [ ]:
#to scale the data 
scaler = MinMaxScaler()
data=train.drop(['TXN_AMNT', 'TARGET'], axis='columns')
data_rescaled = scaler.fit_transform(data)

Principal Component Analysis

In [ ]:
#to select the number of components
pca = PCA().fit(data_rescaled)
plt.plot(pca.explained_variance_, linewidth=2)
plt.xlabel('Components')
plt.ylabel('Explained Variances')
plt.show()  

#PCA
pca = prince.PCA(
     n_components=110,
     n_iter=3,
     rescale_with_mean=True,
     rescale_with_std=True,
     copy=True,
     check_input=True,
     engine='auto',
     random_state=42)
pca = pca.fit(data_rescaled)
pca.transform(data_rescaled).head() 
pca.eigenvalues_
principalComponents = pca.fit_transform(data_rescaled)
train_pc = pd.DataFrame(data = principalComponents)
train_pc['TARGET'] = train['TARGET']
train_pc['TXN_AMNT'] = train['TXN_AMNT']
train_pc.head()

Normalize ‘Amount’

In [ ]:
train_pc['TXN_AMNT(Normalized)'] = StandardScaler().fit_transform(train_pc['TXN_AMNT'].values.reshape(-1,1))
train_pc = train_pc.drop(columns = ['TXN_AMNT'], axis=1) # This column is not necessary anymore.
train_pc.head()

Learning Curve

In [ ]:
y = train_pc["TARGET"]
X = train_pc.drop(['TARGET'], axis='columns')
#
# Create training and test split
#
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=1)
#
# Create a pipeline; This will be passed as an estimator to learning curve method
#
pipeline = make_pipeline(StandardScaler(),
                        LogisticRegression(penalty='l2', solver='lbfgs', random_state=1, max_iter=10000))
#
# Use learning curve to get training and test scores along with train sizes
#
train_sizes, train_scores, test_scores = learning_curve(estimator=pipeline, X=X_train, y=y_train,
                                                       cv=10, train_sizes=np.linspace(0.1, 1.0, 10),
                                                     n_jobs=1)
#
# Calculate training and test mean and std
#
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
#
# Plot the learning curve
#
plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training Accuracy')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
plt.plot(train_sizes, test_mean, color='green', marker='+', markersize=5, linestyle='--', label='Validation Accuracy')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
plt.title('Learning Curve')
plt.xlabel('Training Data Size')
plt.ylabel('Model accuracy')
plt.grid()
plt.legend(loc='upper right')
plt.show()

Train-Test Split

In [ ]:
X = train_pc.drop('TARGET', axis=1)
y = train_pc['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# We are transforming data to numpy array to implementing with keras
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
# MODELING

# 1. Decision Tree
print('Decision Tree')
tree_model = DecisionTreeClassifier(max_depth = 4, criterion = 'entropy')
tree_model.fit(X_train, y_train)
tree_yhat = tree_model.predict(X_test)

# 3. Logistic Regression
print('Logistic Regression')
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
lr.fit(X_train, y_train)
lr_yhat = lr.predict(X_test)


# 5. Random Forest Tree
print('Random Forest Tree')
rf = RandomForestClassifier(max_depth = 4)
rf.fit(X_train, y_train)
rf_yhat = rf.predict(X_test)

# 6. XGBoost
print('XGBoost')
xgb = XGBClassifier(max_depth = 4)
xgb.fit(X_train, y_train)
xgb_yhat = xgb.predict(X_test)

In [ ]:
# 1. Accuracy score

print(cl('ACCURACY SCORE', attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the Decision Tree model is {}'.format(accuracy_score(y_test, tree_yhat)), attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the Logistic Regression model is {}'.format(accuracy_score(y_test, lr_yhat)), attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the Random Forest Tree model is {}'.format(accuracy_score(y_test, rf_yhat)), attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the XGBoost model is {}'.format(accuracy_score(y_test, xgb_yhat)), attrs = ['bold'], color = 'red'))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))

In [ ]:
#Artificial Neural Networks
from keras.models import Sequential
from keras.layers import Dense, Dropout
model = Sequential([
    Dense(units=20, input_dim = X_train.shape[1], activation='relu'),
    Dense(units=24,activation='relu'),
    Dropout(0.5),
    Dense(units=20,activation='relu'),
    Dense(units=24,activation='relu'),
    Dense(1, activation='sigmoid')
])
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=30, epochs=5)
score = model.evaluate(X_test, y_test)
print('Test Accuracy: {:.2f}%\nTest Loss: {}'.format(score[1]*100,score[0]))

In [ ]:
# Count the occurrences of fraud and no fraud and print them
occ = train_pc['TARGET'].value_counts()
occ
# Print the ratio of fraud cases
ratio_cases = occ/len(train_pc.index)
print(f'Ratio of fraudulent cases: {ratio_cases[1]}\nRatio of non-fraudulent cases: {ratio_cases[0]}') # class imbalance problem

# SMOTE Sampling

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, classification_report
X_smote, y_smote = SMOTE().fit_resample(X, y)
X_smote = pd.DataFrame(X_smote)
y_smote = pd.DataFrame(y_smote)
y_smote.iloc[:,0].value_counts()
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.3, random_state=0)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size = 30, epochs = 5)
score = model.evaluate(X_test, y_test)
print('Test Accuracy: {:.2f}%\nTest Loss: {}'.format(score[1]*100,score[0]))
y_pred = model.predict(X_test)
y_test = pd.DataFrame(y_test)
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred.round())
sns.heatmap(cm, annot=True, fmt='.0f')
plt.show()

In [ ]:
# Count the occurrences of fraud and no fraud and print them
occ = y_smote.value_counts()
occ
# Print the ratio of fraud cases
ratio_cases = occ/len(y_smote.index)
print(f'Ratio of fraudulent cases: {ratio_cases[1]}\nRatio of non-fraudulent cases: {ratio_cases[0]}') # problem solved

In [ ]:
y_pred2 = model.predict(X_test)
y_test2 = pd.DataFrame(y_test)
from sklearn.metrics import confusion_matrix, classification_report
cm2 = confusion_matrix(y_test2, y_pred2.round())
sns.heatmap(cm2, annot=True, fmt='.0f', cmap='coolwarm')
plt.show()

# plt
fpr, tpr, threshold = metrics.roc_curve(y_test2, y_pred2.round())
roc_auc = metrics.auc(fpr, tpr)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Modeling

In [ ]:
# 1. Decision Tree
print('Decision Tree')
tree_model = DecisionTreeClassifier(max_depth = 4, criterion = 'entropy')
tree_model.fit(X_train, y_train)
tree_yhat = tree_model.predict(X_test)

# 2. Logistic Regression
print('Logistic Regression')
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
lr.fit(X_train, y_train)
lr_yhat = lr.predict(X_test)


# 3. Random Forest Tree
print('Random Forest Tree')
rf = RandomForestClassifier(max_depth = 4)
rf.fit(X_train, y_train)
rf_yhat = rf.predict(X_test)

# 4. XGBoost
print('XGBoost')
xgb = XGBClassifier(max_depth = 4)
xgb.fit(X_train, y_train)
xgb_yhat = xgb.predict(X_test)

# 5. ANN
print('ANN')
model.fit(X_train, y_train, batch_size = 30, epochs = 5)
ann_pred = model.predict(X_test)

# 6. Blending model
print('Blending model')
blended= 0.1*y_pred2.round()+(0.5*knn_yhat+ 0.1*xgb_yhat +0.1*rf_yhat+0.1*lr_yhat+0.1*tree_yhat).reshape(320979,1)

 Accuracy Score

In [ ]:
print(cl('ACCURACY SCORE', attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the Decision Tree model is {}'.format(accuracy_score(y_test, tree_yhat)), attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the Logistic Regression model is {}'.format(accuracy_score(y_test, lr_yhat)), attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the Random Forest Tree model is {}'.format(accuracy_score(y_test, rf_yhat)), attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the XGBoost model is {}'.format(accuracy_score(y_test, xgb_yhat)), attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the ANN model is {}'.format(accuracy_score(y_test, ann_pred.round())), attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the Blending model is {}'.format(accuracy_score(y_test, blended.round())), attrs = ['bold'], color = 'red'))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))

# Tuning Hyperparameters of An Artificial Neural Network Leveraging Keras Tuner

In [ ]:
msle = MeanSquaredLogarithmicError()

def build_model(hp):
  model = tf.keras.Sequential()
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
  hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
  hp_units3 = hp.Int('units3', min_value=32, max_value=512, step=32)
  model.add(Dense(units=hp_units1, activation='relu'))
  model.add(tf.keras.layers.Dense(units=hp_units2, activation='relu'))
  model.add(tf.keras.layers.Dense(units=hp_units3, activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='linear'))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2,1e-3,1e-4])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
      loss=msle,
      metrics=[msle]
  )

  return model

# HyperBand algorithm from keras tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_mean_squared_logarithmic_error',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)

tuner.search(X_train, y_train, epochs=10, validation_split=0.2) 


#Trial 31 Complete [00h 03m 08s]
#val_mean_squared_logarithmic_error: 0.03584810346364975

#Best val_mean_squared_logarithmic_error So Far: 0.024163799360394478
#Total elapsed time: 00h 52m 05s
#INFO:tensorflow:Oracle triggered exit

In [ ]:
for h_param in [f"units{i}" for i in range(1,4)] + ['learning_rate']:
  print(h_param, tuner.get_best_hyperparameters()[0].get(h_param))

#units1 192
#units2 352
#units3 352
#learning_rate 0.0001

In [ ]:
best_model = tuner.get_best_models()[0]
best_model.fit(
    X_train, 
    y_train,
    epochs=10,
    batch_size=30
)

# XGBoost Hyperparameter Tuning Using Optuna

In [ ]:
#XGBoost Regressor Optuna
def xgbobj(trial , data = X_smote , target = y_smote):
        params = {
        "n_estimators": 100,
        "subsample": trial.suggest_discrete_uniform("subsample",0.8,1,0.1),
        "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree",0.3,0.8,0.1),
        "eta": 0.01,
        "reg_alpha": trial.suggest_int("reg_alpha",0,30,1),
        "reg_lambda": trial.suggest_int("reg_lambda",0,1,0.1),
        "max_depth": trial.suggest_int("max_depth",3,5,1),
        "min_child_weight": trial.suggest_int("min_child_weight",1,3,1),
    }
    model = xgb.XGBRegressor(**params, tree_method = "gpu_hist",random_state=42)
    model.fit(X_train,y_train,eval_set = [(X_test,y_test)] ,early_stopping_rounds=100, verbose = False , eval_metric = "auc")

    y_preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_preds))
    return rmse

In [ ]:
study = optuna.create_study(direction = "minimize")
study.optimize(xgbobj, n_trials = 15)
print("Number of finished trials:", len(study.trials))
print("Best trial: score {}, params {}".format(study.best_trial.value, study.best_trial.params))


#Number of finished trials: 15
#Best trial: score 0.3579398863954667, params {'subsample': 0.8, 'colsample_bytree': 0.7, 'reg_alpha': 25, 
#'reg_lambda': 0, 'max_depth': 5, 'min_child_weight': 2}

In [ ]:
#XGBoost with Optuna Hyperparameters
modelxgb = xgb.XGBRegressor(n_estimators= 100, tree_method='gpu_hist', subsample= 0.8, colsample_bytree= 0.7,
       eta= 0.01, reg_alpha= 25, reg_lambda= 0, max_depth= 5, min_child_weight= 2).fit(X_train , y_train)


xgbpred = np.expm1(modelxgb.predict(X_test))
y_test = pd.DataFrame(y_test)

# plt
fpr, tpr, threshold = metrics.roc_curve(y_test, xgbpred)
roc_auc = metrics.auc(fpr, tpr)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Modeling

In [ ]:
# 1. Decision Tree
print('Decision Tree')
tree_model2 = DecisionTreeClassifier(max_depth = 4, criterion = 'entropy')
tree_model2.fit(X_train, y_train)
tree_yhat2 = tree_model2.predict(X_test)


# 2. Logistic Regression
print('Logistic Regression')
lr2 = LogisticRegression(solver='lbfgs', max_iter=1000)
lr2.fit(X_train, y_train)
lr_yhat2 = lr2.predict(X_test)


# 3. Random Forest Tree
print('Random Forest Tree')
rf2 = RandomForestClassifier(max_depth = 4)
rf2.fit(X_train, y_train)
rf_yhat2 = rf2.predict(X_test)

# 4. XGBoost with Optuna Hyperparameters
print('XGBoost')
xgbpred = np.expm1(modelxgb.predict(X_test))


# 5.Artificial Neural Networks with Keras Tuner
ann_pred = tf.keras.activations.sigmoid(best_model.predict(X_test))
ann_pred = np.where(pd.DataFrame(ann_pred.numpy())>0.5, 1, 0) 


# 6.Blending Model
blended=0.1*ann_pred+(0.15*xgb_yhat2+0.25*rf_yhat2+0.25*lr_yhat2+0.25*tree_yhat2).reshape(320979,1)
blended=np.where(pd.DataFrame(blended)>0.5, 1, 0) 

Accuracy Score

In [ ]:
print(cl('ACCURACY SCORE', attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the Decision Tree model is {}'.format(accuracy_score(y_test, tree_yhat2)), attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the Logistic Regression model is {}'.format(accuracy_score(y_test, lr_yhat2)), attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the Random Forest Tree model is {}'.format(accuracy_score(y_test, rf_yhat2)), attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the ANN model is {}'.format(accuracy_score(y_test, ann_pred)), attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the XGB_tuned model is {}'.format(accuracy_score(y_test, xgbpred)), attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the Blended model is {}'.format(accuracy_score(y_test, blended)), attrs = ['bold'], color = 'red'))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))

# Forecasting
Data PreProcessing

In [ ]:
real_test = pd.read_csv("test.csv")
real_test.TXN_TIME=pd.cut(real_test.TXN_TIME,bins=[-1,600,1200,1800,2400],labels=['1','2','3','4'])  
real_test["TXN_TIME"] = real_test["TXN_TIME"].astype('int')
real_test.DAY_OF_MONTH=pd.cut(real_test.DAY_OF_MONTH,bins=[20,30],labels=['4'])
real_test["DAY_OF_MONTH"] = real_test["DAY_OF_MONTH"].astype('int')
real_test["DAY_OF_WEEK"] = real_test["DAY_OF_WEEK"].astype('int')
real_test["TXN_AMNT"] = real_test["TXN_AMNT"].astype('int')
del real_test['TXN_TRM']
del real_test['ID']
real_test.info()
real_test.isna().sum() 
real_test.describe(include='all')
#categorical data
categorical = ['CST_NR', 'CC_NR', 'TXN_SOURCE', 'TXN_ENTRY','CITY','COUNTRY','MC_NAME','MC_ID','MCC_CODE'] 
for feature in categorical:
  real_test[feature] = real_test[feature].mask(real_test[feature].map(real_test[feature].value_counts(normalize=True)) < 0.001,
                                               'Other')

In [ ]:
real_test = pd.get_dummies(real_test, columns = categorical)
real_test.head()


real_test_data=real_test.drop('TXN_AMNT', axis= 1)
real_test_rescaled = scaler.fit_transform(real_test_data)

#PCA
pca = pca.fit(real_test_rescaled)
pca.transform(real_test_rescaled).head() 
pca.eigenvalues_
principalComponents2 = pca.fit_transform(real_test_rescaled)
real_test_pc = pd.DataFrame(data = principalComponents2)
real_test_pc['TXN_AMNT'] = real_test['TXN_AMNT']
real_test_pc.head()


#Normalize ‘Amount’
from sklearn.preprocessing import StandardScaler
real_test_pc['TXN_AMNT(Normalized)'] = StandardScaler().fit_transform(real_test_pc['TXN_AMNT'].values.reshape(-1,1))
real_test_pc = real_test_pc.drop(columns = ['TXN_AMNT'], axis=1) # This column is not necessary anymore.
real_test_pc.head()

#Data PreProcessing
X_train = train_pc.drop('TARGET', axis=1)
y_train = train_pc['TARGET']
X_test = real_test_pc

#SMOTE Sampling
X_train_smote, y_train_smote = SMOTE().fit_resample(X_train, y_train)
X_train_smote = pd.DataFrame(X_train_smote)
y_train_smote = pd.DataFrame(y_train_smote)

 Modeling

In [ ]:

# 1. Decision Tree
print('Decision Tree')
tree_model3 = DecisionTreeClassifier(max_depth = 4, criterion = 'entropy')
tree_model3.fit(X_train_smote, y_train_smote)
tree_yhat3 = tree_model3.predict(X_test)

# 2. Logistic Regression
print('Logistic Regression')
lr3 = LogisticRegression(solver='lbfgs', max_iter=1000)
lr3.fit(X_train_smote, y_train_smote)
lr_yhat3 = lr3.predict(X_test)


# 3. Random Forest Tree
print('Random Forest Tree')
rf3 = RandomForestClassifier(max_depth = 4)
rf3.fit(X_train_smote, y_train_smote)
rf_yhat3 = rf3.predict(X_test)

# 4. XGBoost with Optuna Hyperparameters
print('XGBoost')
modelxgb2 = xgb.XGBRegressor(n_estimators= 100, tree_method='gpu_hist', subsample= 0.8, colsample_bytree= 0.7,
       eta= 0.01, reg_alpha= 25, reg_lambda= 0, max_depth= 5, min_child_weight= 2).fit(X_train_smote,y_train_smote)
xgbpred2 = np.expm1(modelxgb2.predict(X_test))


# 5.Artificial Neural Networks with Keras Tuner
print('Artificial Neural Networks')
best_model = tuner.get_best_models()[0]
best_model.fit(
    X_train_smote, 
    y_train_smote,
    epochs=10,
    batch_size=30
)
ann_pred2 = tf.keras.activations.sigmoid(best_model.predict(X_test))


# 6.Blending Model
print('Blending Model')
blended2= 0.1*ann_pred2+(0.15*xgbpred2+0.25*rf_yhat3+0.25*lr_yhat3+0.25*tree_yhat3).reshape(326232,1) 


In [ ]:
submit = pd.read_csv("sample_submission.csv")
submit.iloc[:,1]=pd.DataFrame(blended2)
submit.to_csv("late3_submit.csv", index=False)